In [65]:
import pandas as pd
import pickle
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt

In [66]:
# Cargar el modelo guardado
with open('mejor_modelo_calidad_lead.pkl', 'rb') as file:
    mejor_modelo = pickle.load(file)

In [67]:
# Definir una función para cargar el modelo y calcular el score
def calcular_puntaje(df):
    # Preprocesar los datos
    X = df.drop(['Calidad_de_lead__c'], axis=1)
    
    # Normalizar los datos
    escalador = StandardScaler()
    X_normalizado = escalador.fit_transform(X)
    
    # Predecir usando el modelo
    puntajes_raw = mejor_modelo.predict(X_normalizado)
    
    # Obtener la importancia de las características
    importancias = mejor_modelo.feature_importances_
    
    # Calcular los puntajes basados en la importancia de las características
    puntajes_importancia = X.dot(importancias)
    
    # Escalar las puntuaciones entre 0 y 100
    escalador_minmax = MinMaxScaler(feature_range=(0, 100))
    puntajes_importancia_escalados = escalador_minmax.fit_transform(puntajes_importancia.values.reshape(-1, 1))
    
    # Agregar una nueva columna con los puntajes al DataFrame original
    df['Puntaje_calculado'] = puntajes_importancia_escalados
    
    return df

In [68]:
df = pd.read_csv('Datos_Train_Calidad_Lead_Clean.csv')
df.head()

,pctcuota_inicial__c,Presupuesto_inmueble_a_comprar__c,de_ingresos_sobre_el_inmueble__c,picklist_ciudad__c,Calidad_de_lead__c
0,2.330000,3.000000e+08,1.71,1,-1
1,1.000000,2.000000e+08,1.41,1,-1
2,4405.569989,2.487111e+08,-20.00,1,-1
3,0.000000,1.800000e+07,10.00,1,-1
4,0.000000,1.700000e+08,1.76,1,-1


In [72]:
# Filtrar los registros que tienen el valor 0 en la columna 'Calidad_de_lead__c'
filtro = df['Calidad_de_lead__c'] == -1

# Contar los registros que cumplen con el filtro
conteo = df[filtro].shape[0]

print(f'Número de registros con Calidad_de_lead__c igual a 0: {conteo}')

Número de registros con Calidad_de_lead__c igual a 0: 116


In [70]:
df_con_puntajes = calcular_puntaje(df)
df_con_puntajes.head()

,pctcuota_inicial__c,Presupuesto_inmueble_a_comprar__c,de_ingresos_sobre_el_inmueble__c,picklist_ciudad__c,Calidad_de_lead__c,Puntaje_calculado
0,2.330000,3.000000e+08,1.71,1,-1,5.730402
1,1.000000,2.000000e+08,1.41,1,-1,3.820268
2,4405.569989,2.487111e+08,-20.00,1,-1,4.750991
3,0.000000,1.800000e+07,10.00,1,-1,0.343825
4,0.000000,1.700000e+08,1.76,1,-1,3.247228


In [74]:
# Filtrar los registros que tienen valores entre 50 y 74
filtro = (df['Puntaje_calculado'] >= 0) & (df['Puntaje_calculado'] <= 25)

# Contar los registros que cumplen con el filtro
conteo = df[filtro].shape[0]

print(f'Número de registros con Puntaje_calculado entre 50 y 74: {conteo}')

Número de registros con Puntaje_calculado entre 50 y 74: 702
